In [2]:
import requests
import pg8000
from shapely import wkb



In [3]:

r = requests.get('http://api.geonames.org/search?featureClass=P&country=FR&maxRows=10&orderby=population&type=json&username=yoannbouge'); 

result = r.json()['geonames']
result

[{'adminCode1': '11',
  'lng': '2.3488',
  'geonameId': 2988507,
  'toponymName': 'Paris',
  'countryId': '3017382',
  'fcl': 'P',
  'population': 2138551,
  'countryCode': 'FR',
  'name': 'Paris',
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': 'IDF'},
  'countryName': 'France',
  'fcodeName': 'capital of a political entity',
  'adminName1': 'Île-de-France',
  'lat': '48.85341',
  'fcode': 'PPLC'},
 {'adminCode1': '93',
  'lng': '5.38107',
  'geonameId': 2995469,
  'toponymName': 'Marseille',
  'countryId': '3017382',
  'fcl': 'P',
  'population': 794811,
  'countryCode': 'FR',
  'name': 'Marseille',
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': 'PAC'},
  'countryName': 'France',
  'fcodeName': 'seat of a first-order administrative division',
  'adminName1': "Provence-Alpes-Côte d'Azur",
  'lat': '43.29695',
  'fcode': 'PPLA'},
 {'adminCode1': '84',
  'lng': '4.84671',
  'geonameId': 2996944,
  'toponymName': 'Lyon',
  'countryId': '3017382',
  'fcl

In [4]:
resu = list(map(lambda x: {'geonameId': x['geonameId'], 'name': x[
            'name'], 'geometry': (x['lng']) + ' ' + x['lat']}, result))
resu

[{'geonameId': 2988507, 'name': 'Paris', 'geometry': '2.3488 48.85341'},
 {'geonameId': 2995469, 'name': 'Marseille', 'geometry': '5.38107 43.29695'},
 {'geonameId': 2996944, 'name': 'Lyon', 'geometry': '4.84671 45.74846'},
 {'geonameId': 2972315, 'name': 'Toulouse', 'geometry': '1.44367 43.60426'},
 {'geonameId': 2990440, 'name': 'Nice', 'geometry': '7.26608 43.70313'},
 {'geonameId': 2990969, 'name': 'Nantes', 'geometry': '-1.55336 47.21725'},
 {'geonameId': 2973783, 'name': 'Strasbourg', 'geometry': '7.74553 48.58392'},
 {'geonameId': 2992166, 'name': 'Montpellier', 'geometry': '3.87635 43.61093'},
 {'geonameId': 3031582, 'name': 'Bordeaux', 'geometry': '-0.5805 44.84044'},
 {'geonameId': 2998324, 'name': 'Lille', 'geometry': '3.05858 50.63297'}]

In [24]:
import ssl
ssl_context = ssl.SSLContext()
ssl_context.verify_mode = ssl.CERT_NONE
conn = pg8000.connect(user = "p1711842", host = "bd-pedago.univ-lyon1.fr", port = 5432,
                              database = "p1711842", password = "Pu1WtY83xbUx", ssl_context=ssl_context)
localConnection = pg8000.connect(user = "postgres", host = "localhost", port = 5432,
                              database = "postgres", password = "password")

In [48]:

for city in resu:
    conn.run("INSERT INTO public.villes2(geonameid, geom, name) Values({}, {}, '{}');".format(
        city['geonameId'], "ST_GeomFromText('POINT({})', 4326)".format(city['geometry']), city['name']))
conn.commit()

In [49]:
"""
CREATE TABLE ville_distance_min AS 
SELECT vi.geonameid, vi.name, vi.geom, min(ST_DistanceSphere(ae.geom, vi.geom)) as dist_meters
	FROM  public.villes as vi, public.meteo_aeroports as ae GROUP BY vi.geonameid, vi.name, vi.geom;

SELECT vi.geom, vi.geonameid, ae.city, ae.icao, ST_DistanceSphere(ae.geom, vi.geom)
	FROM public.ville_distance_min as vi, public.meteo_aeroports as ae WHERE ST_DistanceSphere(ae.geom, vi.geom)=dist_meters;
"""


"""
Test : 
SELECT vi.geom, vi.geonameid, ae.city, ae.icao, ST_DistanceSphere(ae.geom, vi.geom),ae.airport_name
	FROM public.ville_distance_min2 as vi, public.meteo_aeroports2 as ae WHERE ST_DistanceSphere(ae.geom, vi.geom)=dist_meters;

CREATE TABLE aeroport_proche_ville2 AS 
SELECT vi.geom, vi.geonameid, ae.city, ae.icao, ST_DistanceSphere(ae.geom, vi.geom),ae.airport_name
	FROM public.ville_distance_min2 as vi, public.meteo_aeroports2 as ae WHERE ST_DistanceSphere(ae.geom, vi.geom)=dist_meters;


"""


conn.close()

In [7]:
villes = conn.run("SELECT * from public.villes")
villes

([2988507, '0101000020E610000027C286A757CA0240280AF4893C6D4840', 'Paris'],
 [2995469, '0101000020E610000067F2CD36378615400B46257502A64540', 'Marseille'],
 [2996944, '0101000020E6100000DEC83CF2076313409D2E8B89CDDF4640', 'Lyon'],
 [2972315, '0101000020E61000000C76C3B64519F73FF623456458CD4540', 'Toulouse'],
 [2990440, '0101000020E61000008D7A884677101D40126BF12900DA4540', 'Nice'],
 [2990969, '0101000020E61000000AA2EE0390DAF8BF2B8716D9CE9B4740', 'Nantes'],
 [2973783, '0101000020E61000005DBF60376CFB1E407BBDFBE3BD4A4840', 'Strasbourg'],
 [2992166,
  '0101000020E6100000FBCBEEC9C3020F40991249F432CE4540',
  'Montpellier'],
 [3031582, '0101000020E6100000FA7E6ABC7493E2BF0820B589936B4640', 'Bordeaux'],
 [2998324, '0101000020E6100000F29881CAF8770840B0AC342905514940', 'Lille'])

In [8]:
villesX = list(map(lambda x: [*x[0:1],wkb.loads(x[1], hex=True),*x[2:]],
                     villes))
villesX

[[2988507, <shapely.geometry.point.Point at 0x7f0c08d23790>, 'Paris'],
 [2995469, <shapely.geometry.point.Point at 0x7f0c08d23ca0>, 'Marseille'],
 [2996944, <shapely.geometry.point.Point at 0x7f0c08d23e50>, 'Lyon'],
 [2972315, <shapely.geometry.point.Point at 0x7f0c0415d610>, 'Toulouse'],
 [2990440, <shapely.geometry.point.Point at 0x7f0c04157040>, 'Nice'],
 [2990969, <shapely.geometry.point.Point at 0x7f0c08d23610>, 'Nantes'],
 [2973783, <shapely.geometry.point.Point at 0x7f0c08d23cd0>, 'Strasbourg'],
 [2992166, <shapely.geometry.point.Point at 0x7f0c08d23a30>, 'Montpellier'],
 [3031582, <shapely.geometry.point.Point at 0x7f0c08d23af0>, 'Bordeaux'],
 [2998324, <shapely.geometry.point.Point at 0x7f0c08d231f0>, 'Lille']]

In [10]:
aeroports = conn.run("SELECT * from public.airports")
aeroports

([1254,
  '0101000020E6100000CC5EB69DB646FF3F344A97FE257B4940',
  'Calais Dunkerque',
  'Calais',
  'France',
  'CQF',
  'LFAC',
  50.962097,
  1.954764,
  12,
  1.0,
  'E',
  'Europe/Paris'],
 [1255,
  '0101000020E6100000DE72F563933C0840FB04508C2CEF4840',
  'Peronne St Quentin',
  'Peronne',
  'France',
  None,
  'LFAG',
  49.868547,
  3.029578,
  295,
  1.0,
  'E',
  'Europe/Paris'],
 [1256,
  '0101000020E6100000B49080D1E50D08407D2079E7504C4840',
  'Les Loges',
  'Nangis',
  'France',
  None,
  'LFAI',
  48.596219,
  3.006786,
  428,
  1.0,
  'E',
  'Europe/Paris'],
 [1257,
  '0101000020E6100000DA0242EBE1CBD8BF5F7D3CF4DD454840',
  'Couterne',
  "Bagnole-de-l'orne",
  'France',
  None,
  'LFAO',
  48.545836,
  -0.387444,
  718,
  1.0,
  'E',
  'Europe/Paris'],
 [1258,
  '0101000020E6100000A88E554ACF9405406C06B8205BFC4840',
  'Bray',
  'Albert',
  'France',
  None,
  'LFAQ',
  49.971531,
  2.697661,
  364,
  1.0,
  'E',
  'Europe/Paris'],
 [1259,
  '0101000020E6100000FE2955A2ECEDF93F71

In [12]:
aeroportsX = list(map(lambda x: [*x[0:1],wkb.loads(x[1], hex=True),*x[2:]],
                     aeroports))
aeroportsX

[[1254,
  'Calais Dunkerque',
  'Calais',
  'France',
  'CQF',
  'LFAC',
  50.962097,
  1.954764,
  12,
  1.0,
  'E',
  'Europe/Paris'],
 [1255,
  'Peronne St Quentin',
  'Peronne',
  'France',
  None,
  'LFAG',
  49.868547,
  3.029578,
  295,
  1.0,
  'E',
  'Europe/Paris'],
 [1256,
  'Les Loges',
  'Nangis',
  'France',
  None,
  'LFAI',
  48.596219,
  3.006786,
  428,
  1.0,
  'E',
  'Europe/Paris'],
 [1257,
  'Couterne',
  "Bagnole-de-l'orne",
  'France',
  None,
  'LFAO',
  48.545836,
  -0.387444,
  718,
  1.0,
  'E',
  'Europe/Paris'],
 [1258,
  'Bray',
  'Albert',
  'France',
  None,
  'LFAQ',
  49.971531,
  2.697661,
  364,
  1.0,
  'E',
  'Europe/Paris'],
 [1259,
  'Le Touquet Paris Plage',
  'Le Tourquet',
  'France',
  'LTQ',
  'LFAT',
  50.517385,
  1.620587,
  36,
  1.0,
  'E',
  'Europe/Paris'],
 [1260,
  'Denain',
  'Valenciennes',
  'France',
  None,
  'LFAV',
  50.325808,
  3.461264,
  177,
  1.0,
  'E',
  'Europe/Paris'],
 [1261,
  'Glisy',
  'Amiens',
  'France',
  N

In [19]:
conn.run("CREATE TABLE if not exists public.aireports_proche_villes (geonameid INT, geom Geometry, airport_name character varying, distance INT, icao character varying, city character varying);")
conn.commit()

In [25]:
import math

def distance_haversine(p1, p2): 
	lon1 = p1.x if 'x' in dir(p1) else p1.lon if 'lon' in dir(p1) else p1['lon'] if 'lon' in p1 else p1[0]
	lat1 = p1.y if 'y' in dir(p1) else p1.lat if 'lat' in dir(p1) else p1['lat'] if 'lat' in p1 else p1[1]
	lon2 = p2.x if 'x' in dir(p2) else p2.lon if 'lon' in dir(p2) else p2['lon'] if 'lon' in p2 else p2[0]
	lat2 = p2.y if 'y' in dir(p2) else p2.lat if 'lat' in dir(p2) else p2['lat'] if 'lat' in p2 else p2[1]
	lonr1 = math.radians(lon1)
	latr1 = math.radians(lat1)
	lonr2 = math.radians(lon2)
	latr2 = math.radians(lat2)
	return 2 * 6378137 * math.sin(math.sqrt(math.pow(math.sin((latr2 - latr1) / 2.0), 2) + math.cos(latr1) * math.cos(latr2) * math.pow(math.sin((lonr2 - lonr1) / 2.0), 2)))

for ville in villesX :
    min = 99999999999999999
    aero = {}
    for aeroport in aeroportsX :
        if min > distance_haversine(aeroport[1],ville[1]) :
            min = distance_haversine(aeroport[1],ville[1])
            aero = aeroport
    conn.run("insert into public.aireports_proche_villes(geom,geonameid,city,icao, distance,airport_name) values('{}', {},'{}','{}',{},'{}')".format(
        wkb.dumps(ville[1], hex=True, srid=4326),ville[0], ville[2],aero[6],min,aero[2].replace("\\'","''")))
conn.commit()
